In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(nu, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta, N_series):
    a =  prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    tt = np.arange(0.001, max(RT)+0.02, 0.02)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

N_series = 250

In [6]:
for n in tqdm(range(150)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta, N_series), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
recovery_df = pd.DataFrame(recovery_df)

100%|██████████████████████████████████| 150/150 [6:29:01<00:00, 155.61s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,0.718888,0.726237,0.465679,0.470679,2.279882,2.076927,-1.963508,-2.030856
1,1.756578,1.778310,0.868142,0.876824,1.352574,1.384509,-1.969463,-2.037989
2,2.178532,2.192024,0.908898,0.900617,1.647686,1.660954,-2.694208,-2.642728
3,4.409969,5.000000,0.561883,0.596559,2.590516,3.050951,-2.887183,-3.500000
4,2.109966,1.962997,0.970462,1.000000,2.224068,2.062646,-2.757606,-2.748335
...,...,...,...,...,...,...,...,...
145,3.284020,3.310996,0.781482,0.774225,-1.490150,-1.486156,0.136519,0.201389
146,3.293142,3.034454,0.526217,0.613431,3.228604,3.325564,0.719394,0.709229
147,3.035715,2.758261,0.623280,0.735657,-1.346806,-1.348673,-0.025638,-0.059128
148,3.524767,2.559531,0.685724,1.000000,2.343650,2.146387,-0.499802,-0.491233


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.909955,-0.068001,-0.000532,-0.107914,-0.107538,0.170075,0.131662
threshold_estimate,0.909955,1.000000,-0.076061,-0.205849,-0.090780,-0.093011,0.122674,0.108103
ndt_true,-0.068001,-0.076061,1.000000,0.830117,-0.049999,-0.059359,-0.002511,0.030857
ndt_estimate,-0.000532,-0.205849,0.830117,1.000000,-0.032141,-0.044230,0.035439,0.042434
mu1_true,-0.107914,-0.090780,-0.049999,-0.032141,1.000000,0.990124,-0.038527,-0.068135
mu1_estimate,-0.107538,-0.093011,-0.059359,-0.044230,0.990124,1.000000,-0.031165,-0.056931
mu2_true,0.170075,0.122674,-0.002511,0.035439,-0.038527,-0.031165,1.000000,0.959966
mu2_estimate,0.131662,0.108103,0.030857,0.042434,-0.068135,-0.056931,0.959966,1.000000


In [9]:
recovery_df.to_csv('Series_2d_recovery_{}.csv'.format(N_series))